## Data Importing and Cleaning

In [1]:
# Import the libraries
import pandas as pd

In [2]:
# Read in the dataset
df = pd.read_sas('../data/LLCP2017.XPT', encoding='utf-8')

/opt/conda/lib/python3.7/site-packages/pandas/io/sas/sas_xport.py:475: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[x] = v


In [3]:
# check number of rows and columns
df.shape

(450016, 358)

In [4]:
# check the dataframe
df.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_PA300R2,_PA30021,_PASTRNG,_PAREC1,_PASTAE1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,1.0,1.0,01302017,01,30,2017,1100.0,2017000001,2.017000e+09,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,01122017,01,12,2017,1100.0,2017000002,2.017000e+09,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,01102017,01,10,2017,1100.0,2017000003,2.017000e+09,1.0,...,3.0,2.0,2.0,4.0,2.0,1.0,1.0,1.0,1.0,2.0
3,1.0,1.0,02082017,02,08,2017,1200.0,2017000004,2.017000e+09,1.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN
4,1.0,1.0,01302017,01,30,2017,1100.0,2017000005,2.017000e+09,1.0,...,3.0,2.0,2.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0


In [5]:
# select columns for analysis after screening through the codebook
df_sel = df[['DIABETE3','_BMI5', 'PHYSHLTH','MENTHLTH', '_MICHD', '_RFCHOL1', '_RFHYPE5', 'PA1VIGM_', 'GRENDA1_', 'POTADA1_', 'FRNCHDA_',
            'FRUTDA2_', '_RFDRHV5', '_RFSMOK3', '_INCOMG', '_EDUCAG', 'SEX', '_AGE65YR']]

In [6]:
df_sel.head()

,DIABETE3,_BMI5,PHYSHLTH,MENTHLTH,_MICHD,_RFCHOL1,_RFHYPE5,PA1VIGM_,GRENDA1_,POTADA1_,FRNCHDA_,FRUTDA2_,_RFDRHV5,_RFSMOK3,_INCOMG,_EDUCAG,SEX,_AGE65YR
0,1.0,2696.0,88.0,88.0,2.0,2.0,2.0,1.800000e+02,43.0,1.400000e+01,1.400000e+01,200.0,1.0,1.0,4.0,4.0,2.0,2.0
1,3.0,2943.0,88.0,88.0,2.0,1.0,2.0,5.397605e-79,100.0,5.397605e-79,5.397605e-79,300.0,1.0,1.0,5.0,4.0,1.0,2.0
2,3.0,2504.0,88.0,88.0,2.0,2.0,1.0,NaN,14.0,5.397605e-79,2.900000e+01,29.0,1.0,1.0,9.0,1.0,1.0,2.0
3,1.0,2663.0,88.0,88.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,3.0,1.0,1.0,1.0,1.0,2.0,2.0
4,3.0,2330.0,14.0,88.0,2.0,1.0,1.0,NaN,14.0,1.400000e+01,1.000000e+01,100.0,1.0,1.0,1.0,1.0,2.0,2.0


In [7]:
# drop missing values
df_sel = df_sel.dropna()
df_sel.count()

DIABETE3    242014
_BMI5       242014
PHYSHLTH    242014
MENTHLTH    242014
_MICHD      242014
_RFCHOL1    242014
_RFHYPE5    242014
PA1VIGM_    242014
GRENDA1_    242014
POTADA1_    242014
FRNCHDA_    242014
FRUTDA2_    242014
_RFDRHV5    242014
_RFSMOK3    242014
_INCOMG     242014
_EDUCAG     242014
SEX         242014
_AGE65YR    242014
dtype: int64

#### After dropping all the rows contains NA or Blanks, we got 239324 samples in the data frame.

In [8]:
## DIABETE3 - check,replace,remove
 # remove 7(don't know) and 9(refused)
 # change order and scale
  # 1 >> 1 (Yes diabetes)
  # 2 >> 1 (Yes Pre- or borderline diabetes)
  # 3 >> 0 (No)
  # 4 >> 1 (Yes Pre- or borderline diabetes)
df_sel['DIABETE3'].value_counts()
df_sel = df_sel[df_sel['DIABETE3'] != 7]
df_sel = df_sel[df_sel['DIABETE3'] != 9]
df_sel['DIABETE3'] = df_sel['DIABETE3'].astype(int)
df_sel['DIABETE3'].replace({1:1,2:1,3:0,4:1},inplace=True)

In [9]:
df_sel['DIABETE3'].value_counts()

0    207380
1     34461
Name: DIABETE3, dtype: int64

In [10]:
##  _BMI5 - check,replace,remove
 # THe original value is BMI * 100. Therefore, divide every value with 100
df_sel['_BMI5'] = df_sel['_BMI5'].div(100)
df_sel['_BMI5'].value_counts()

26.63    3036
27.46    2289
24.41    2207
27.44    2089
27.12    1889
         ... 
77.78       1
17.86       1
48.24       1
58.30       1
58.54       1
Name: _BMI5, Length: 3157, dtype: int64

In [11]:
## PHYSHLTH - check,replace,remove
 # change 88 to 0 (no bad physical health days)
 # remove 77(don't know) and 99(refused)
df_sel = df_sel[df_sel['PHYSHLTH'] != 77]
df_sel = df_sel[df_sel['PHYSHLTH'] != 99]
df_sel['PHYSHLTH'] = df_sel['PHYSHLTH'].replace({88:0})    
df_sel['PHYSHLTH'].value_counts()

0.0     157763
2.0      14916
30.0     12431
1.0      11310
3.0       8282
5.0       7078
10.0      4845
4.0       4238
7.0       4238
15.0      3947
20.0      2445
14.0      2305
6.0       1158
25.0       921
8.0        702
12.0       514
21.0       458
28.0       321
29.0       152
9.0        150
18.0       147
16.0       100
27.0        80
13.0        74
17.0        71
11.0        61
22.0        60
26.0        53
24.0        48
23.0        31
19.0        16
Name: PHYSHLTH, dtype: int64

In [12]:
## MENTHLTH - check,replace,remove
 # change 88 to 0 (no bad mental health days)
 # remove 77(don't know) and 99(refused)
df_sel = df_sel[df_sel['MENTHLTH'] != 77]
df_sel = df_sel[df_sel['MENTHLTH'] != 99]
df_sel['MENTHLTH'] = df_sel['MENTHLTH'].replace({88:0})    
df_sel['MENTHLTH'].value_counts()

0.0     164720
2.0      12673
30.0      9287
1.0       8602
5.0       8498
3.0       7264
10.0      5897
15.0      5046
4.0       3545
7.0       3210
20.0      2946
14.0      1079
6.0       1019
25.0       965
8.0        617
12.0       408
28.0       278
21.0       172
29.0       143
18.0       110
16.0        92
9.0         90
27.0        78
13.0        71
17.0        70
22.0        60
11.0        50
24.0        41
26.0        37
23.0        35
19.0        12
Name: MENTHLTH, dtype: int64

In [13]:
## _MICHD - check,replace,remove
 # change order, scale
  # 1 >> 1 (Reported having MI or CHD)
  # 2 >> 0 (Did not report having MI or CHD)
df_sel['_MICHD'].replace({1:1,2:0},inplace=True)
df_sel['_MICHD'] = df_sel['_MICHD'].astype(int)
df_sel['_MICHD'].value_counts()

0    218494
1     18621
Name: _MICHD, dtype: int64

In [14]:
## _RFCHOL1 - check,replace,remove
 # change order, scale
  # 1 >> 0 (No)
  # 2 >> 1 (Yes)
df_sel['_RFCHOL1'].replace({1:0,2:1},inplace=True)  
df_sel['_RFCHOL1'] = df_sel['_RFCHOL1'].astype(int)
df_sel['_RFCHOL1'].value_counts()

0    148032
1     89083
Name: _RFCHOL1, dtype: int64

In [15]:
## _RFHYPE5 - check,replace,remove
 # removing 9(don't know)
 # change order, scale
  # 1 >> 0 (No)
  # 2 >> 1 (Yes)
df_sel=df_sel[df_sel['_RFHYPE5'] != 9]
df_sel['_RFHYPE5'].replace({1:0,2:1},inplace=True) 
df_sel['_RFHYPE5'] = df_sel['_RFHYPE5'].astype(int)
df_sel['_RFHYPE5'].value_counts()

0    144479
1     92338
Name: _RFHYPE5, dtype: int64

In [16]:
## PA1VIGM_ - check,replace,remove
df_sel['PA1VIGM_'] = df_sel['PA1VIGM_'].round(0)
df_sel['PA1VIGM_'].value_counts()

0.0       123940
120.0       6926
180.0       6590
60.0        6450
90.0        5180
           ...  
1517.0         1
908.0          1
3164.0         1
3040.0         1
2920.0         1
Name: PA1VIGM_, Length: 1527, dtype: int64

In [17]:
## GRENDA1_ - check,replace,remove
df_sel['GRENDA1_'] = df_sel['GRENDA1_'].round(0)
df_sel['GRENDA1_'].value_counts()

100.0     54445
43.0      30355
29.0      25312
57.0      19124
14.0      18075
          ...  
329.0         1
183.0         1
140.0         1
1100.0        1
9100.0        1
Name: GRENDA1_, Length: 107, dtype: int64

In [18]:
## POTADA1_ - check,replace,remove
df_sel['POTADA1_'] = df_sel['POTADA1_'].round(0)
df_sel['POTADA1_'].value_counts()

14.0      41065
29.0      31749
0.0       26178
7.0       22594
43.0      19184
          ...  
743.0         1
229.0         1
7000.0        1
1300.0        1
293.0         1
Name: POTADA1_, Length: 84, dtype: int64

In [19]:
## FRNCHDA_ - check,replace,remove
df_sel['FRNCHDA_'] = df_sel['FRNCHDA_'].round(0)
df_sel['FRNCHDA_'].value_counts()

14.0      42397
0.0       41143
7.0       28444
3.0       24862
29.0      24373
          ...  
307.0         1
303.0         1
900.0         1
1400.0        1
240.0         1
Name: FRNCHDA_, Length: 83, dtype: int64

In [20]:
## FRUTDA2_ - check,replace,remove
df_sel['FRUTDA2_'] = df_sel['FRUTDA2_'].round(0)
df_sel['FRUTDA2_'].value_counts()

100.0     74482
200.0     46952
43.0      18233
300.0     18143
29.0      12212
          ...  
280.0         1
207.0         1
4200.0        1
757.0         1
183.0         1
Name: FRUTDA2_, Length: 122, dtype: int64

In [21]:
## _RFDRHV5 - check,replace,remove
  # 1 >> 0 (Not a heavy Drinker)
  # 2 >> 1 (Is Heavy Drinker)
  # Remove 9 >> (Don't know)
df_sel = df_sel[df_sel['_RFDRHV5'] != 9]
df_sel['_RFDRHV5'].replace({1:0,2:1},inplace=True)
df_sel['_RFDRHV5'] = df_sel['_RFDRHV5'].astype(int)
df_sel['_RFDRHV5'].value_counts()

0    219879
1     14238
Name: _RFDRHV5, dtype: int64

In [22]:
##  _RFSMOK3 - check,replace,remove
 # removing 9(don't know)
 # change order, scale
  # 1 >> 0 (No)
  # 2 >> 1 (Yes)
df_sel=df_sel[df_sel['_RFSMOK3'] != 9]
df_sel['_RFSMOK3'].replace({1:0,2:1},inplace=True) 
df_sel['_RFSMOK3'] = df_sel['_RFSMOK3'].astype(int)
df_sel['_RFSMOK3'].value_counts()

0    206179
1     27060
Name: _RFSMOK3, dtype: int64

In [23]:
##  _INCOMG - check,replace,remove
 # removing 9(don't know)
 # change order, scale
 # 1,2,3,4 ->> 0 (<$50,000)
 # 5 ->> 1 (>=$50,000)
df_sel=df_sel[df_sel['_INCOMG'] != 9]
df_sel['_INCOMG'].replace({1:0,2:0,3:0,4:0,5:1},inplace=True) 
df_sel['_INCOMG'] = df_sel['_INCOMG'].astype(int)
df_sel['_INCOMG'].value_counts()

1    119327
0     88483
Name: _INCOMG, dtype: int64

In [24]:
##  _EDUCAG - check,replace,remove
 # removing 9(don't know)
 # 1,2,3 ->> 0 (have not graduated college or technical school)
 # 4 ->> 1 (have graduated college or technical school)
df_sel=df_sel[df_sel['_EDUCAG'] != 9]
df_sel['_EDUCAG'].replace({1:0,2:0,3:0,4:1},inplace=True) 
df_sel['_EDUCAG'] = df_sel['_EDUCAG'].astype(int)
df_sel['_EDUCAG'].value_counts()

0    109510
1     98155
Name: _EDUCAG, dtype: int64

In [25]:
## SEX - check,replace,remove
  #  1 ->> 0 male
  #  2 ->> 1 female
  #  9 ->> refused (remove)
df_sel=df_sel[df_sel['SEX'] != 9]
df_sel['SEX'] = df_sel['SEX'].astype(int)
df_sel['SEX'].replace({1:0,2:1},inplace=True) 
df_sel['SEX'].value_counts()

1    110151
0     97514
Name: SEX, dtype: int64

In [26]:
## _AGE65YR - check,replace,remove
  #  1 ->> 0 18-64
  #  2 ->> 1 >=65
  #  3 ->> unknown (remove)
df_sel=df_sel[df_sel['_AGE65YR'] != 3]
df_sel['_AGE65YR'] = df_sel['_AGE65YR'].astype(int)
df_sel['_AGE65YR'].replace({1:0,2:1},inplace=True) 
df_sel['_AGE65YR'].value_counts()

0    137365
1     70300
Name: _AGE65YR, dtype: int64

In [27]:
df_sel['DIABETE3'].value_counts()

0    178554
1     29111
Name: DIABETE3, dtype: int64

In [28]:
df_sel.head()

,DIABETE3,_BMI5,PHYSHLTH,MENTHLTH,_MICHD,_RFCHOL1,_RFHYPE5,PA1VIGM_,GRENDA1_,POTADA1_,FRNCHDA_,FRUTDA2_,_RFDRHV5,_RFSMOK3,_INCOMG,_EDUCAG,SEX,_AGE65YR
0,1,26.96,0.0,0.0,0,1,1,180.0,43.0,14.0,14.0,200.0,0,0,0,1,1,1
1,0,29.43,0.0,0.0,0,0,1,0.0,100.0,0.0,0.0,300.0,0,0,1,1,0,1
5,0,27.34,6.0,0.0,1,0,1,60.0,29.0,43.0,14.0,2.0,0,1,0,0,0,1
9,0,27.12,2.0,1.0,0,0,1,0.0,100.0,20.0,300.0,700.0,0,0,0,0,0,0
12,1,23.03,5.0,0.0,0,1,1,125.0,71.0,29.0,0.0,57.0,0,0,0,0,1,1


In [29]:
df_sel.columns = ['diabete', 'bmi', 'physhlth', 'menthlth', 'michd', 'hchol', 'hblpr', 'vpa',
                     'grenda', 'potada', 'frnchda', 'frutda', 'hvdr', 'smok', 'incom50', 'cllgr',
                     'sex', 'age65']

In [30]:
## save to CSV
df_sel.to_csv('../data/2017_diabetes_cleaned.csv', index = False)